In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Matplotlib is building the font cache; this may take a moment.


In [6]:
lines_df = pd.read_csv("../flight_data/airlines.csv")
planes_df = pd.read_csv("../flight_data/airplanes.csv")
ports_df = pd.read_csv("../flight_data/airports.csv")
routes_df = pd.read_csv("../flight_data/routes.csv")

      index  Airline ID                                          Name  \
0         0          -1                                       Unknown   
1         1           1                                Private flight   
2         2           2                                   135 Airways   
3         3           3                                 1Time Airline   
4         4           4  2 Sqn No 1 Elementary Flying Training School   
...     ...         ...                                           ...   
6157   6157       21248                                   GX Airlines   
6158   6158       21251                        Lynx Aviation (L3/SSX)   
6159   6159       21268                               Jetgo Australia   
6160   6160       21270                                  Air Carnival   
6161   6161       21317                                 Svyaz Rossiya   

                  Alias IATA ICAO         Callsign         Country Active  
0                    \N    -  NaN              

In [2]:
print("hello")

hello
